In [142]:
import sys
import os
from datetime import datetime
import pandas as pd

# Add the parent directory to sys.path so local modules can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from parser.normalize import dataframe_to_csv, normalize_scraped_data
from parser.parser import parse_irs_data, parse_irs_data_to_dataframe
from scraper.fetch import fetch_irs_data

In [143]:
irs_data = fetch_irs_data()
irs_data

b'<!DOCTYPE html>\n<html  lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">\n  <head>\n    <meta charset="utf-8" />\n<meta name="description" content="See current federal tax brackets and rates based on your income and filing status." />\n<meta name="abstract" content="See current federal tax brackets and rates based on your income and filing status." />\n<link rel="canonical" href="https://www.irs.gov/filing/federal-income-tax-rates-and-brackets" />\n<meta name="robots" content="index, follow" />\n<meta name="rights" content="United States Internal Revenue Services" />\n<meta property="og:image:url" content="https://www.irs.gov/pub/image/logo_smal

In [144]:
parsed_data = parse_irs_data(irs_data)
parsed_data

{'2024 tax rates for a single taxpayer': {'table': {'Tax rate': 'on taxable income from . . .',
   '10%': '$0',
   '12%': '$11,601',
   '22%': '$47,151',
   '24%': '$100,526',
   '32%': '$191,951',
   '35%': '$243,726',
   '37%': '$609,351'}},
 'Married filing jointly or qualifying surviving spouse': {'table': {'Tax rate': 'on taxable income from . . .',
   '10%': '$0',
   '12%': '$23,201',
   '22%': '$94,301',
   '24%': '$201,051',
   '32%': '$383,901',
   '35%': '$487,451',
   '37%': '$731,201'}},
 'Married filing separately': {'table': {'Tax rate': 'on taxable income from . . .',
   '10%': '$0',
   '12%': '$11,601',
   '22%': '$47,151',
   '24%': '$100,526',
   '32%': '$191,951',
   '35%': '$243,726',
   '37%': '$365,601'}},
 'Head of household': {'table': {'Tax rate': 'on taxable income from . . .',
   '10%': '$0',
   '12%': '$16,551',
   '22%': '$63,101',
   '24%': '$100,501',
   '32%': '$191,951',
   '35%': '$243,701',
   '37%': '$609,351'}}}

In [145]:
df = parse_irs_data_to_dataframe(parsed_data)
df.head()

,Header,Rate,Range
0,2024 tax rates for a single taxpayer,Tax rate,on taxable income from . . .
1,2024 tax rates for a single taxpayer,10%,$0
2,2024 tax rates for a single taxpayer,12%,"$11,601"
3,2024 tax rates for a single taxpayer,22%,"$47,151"
4,2024 tax rates for a single taxpayer,24%,"$100,526"


In [146]:
# Singles
singles = df.iloc[1:8, 1:4]
singles

,Rate,Range
1,10%,$0
2,12%,"$11,601"
3,22%,"$47,151"
4,24%,"$100,526"
5,32%,"$191,951"
6,35%,"$243,726"
7,37%,"$609,351"


In [147]:
# married filing jointly
mfj = df.iloc[9:16, 1:4]
mfj

,Rate,Range
9,10%,$0
10,12%,"$23,201"
11,22%,"$94,301"
12,24%,"$201,051"
13,32%,"$383,901"
14,35%,"$487,451"
15,37%,"$731,201"


In [148]:
# married filing separately
mfs = df.iloc[17:24, 1:4]
mfs

,Rate,Range
17,10%,$0
18,12%,"$11,601"
19,22%,"$47,151"
20,24%,"$100,526"
21,32%,"$191,951"
22,35%,"$243,726"
23,37%,"$365,601"


In [149]:
# Head of household
hoh = df.iloc[25:32, 1:4]
hoh

,Rate,Range
25,10%,$0
26,12%,"$16,551"
27,22%,"$63,101"
28,24%,"$100,501"
29,32%,"$191,951"
30,35%,"$243,701"
31,37%,"$609,351"


In [150]:
# add year column to singles, mfj, mfs, hoh
year = datetime.now().year
singles.insert(0, 'Year', year)
mfj.insert(0, 'Year', year)
mfs.insert(0, 'Year', year)
hoh.insert(0, 'Year', year)

singles

,Year,Rate,Range
1,2025,10%,$0
2,2025,12%,"$11,601"
3,2025,22%,"$47,151"
4,2025,24%,"$100,526"
5,2025,32%,"$191,951"
6,2025,35%,"$243,726"
7,2025,37%,"$609,351"


In [151]:
# add a column "" that with the character '>' between the Rate and Range columns
greater_than = '>'
# add the column between Rate and Range columns
singles.insert(2, 'For Income >', greater_than)
mfj.insert(2, 'For Income >', greater_than)
mfs.insert(2, 'For Income >', greater_than)
hoh.insert(2, 'For Income >', greater_than)
singles

,Year,Rate,For Income >,Range
1,2025,10%,>,$0
2,2025,12%,>,"$11,601"
3,2025,22%,>,"$47,151"
4,2025,24%,>,"$100,526"
5,2025,32%,>,"$191,951"
6,2025,35%,>,"$243,726"
7,2025,37%,>,"$609,351"


In [152]:
# rename the rate column 
singles.rename(columns={singles.columns[1]: 'Single Filer (Rates/Brackets)'}, inplace=True)
mfj.rename(columns={mfj.columns[1]: 'Married Filing Jointly (Rates/Brackets)'}, inplace=True)
mfs.rename(columns={mfs.columns[1]: 'Married Filing Separately (Rates/Brackets)'}, inplace=True)
hoh.rename(columns={hoh.columns[1]: 'Head of Household (Rates/Brackets)'}, inplace=True)

singles

,Year,Single Filer (Rates/Brackets),For Income >,Range
1,2025,10%,>,$0
2,2025,12%,>,"$11,601"
3,2025,22%,>,"$47,151"
4,2025,24%,>,"$100,526"
5,2025,32%,>,"$191,951"
6,2025,35%,>,"$243,726"
7,2025,37%,>,"$609,351"


In [153]:
# rename the range column
singles.rename(columns={singles.columns[3]: 'Range Start'}, inplace=True)
mfj.rename(columns={mfj.columns[3]: 'Range Start'}, inplace=True)
mfs.rename(columns={mfs.columns[3]: 'Range Start'}, inplace=True)
hoh.rename(columns={hoh.columns[3]: 'Range Start'}, inplace=True)
singles

,Year,Single Filer (Rates/Brackets),For Income >,Range Start
1,2025,10%,>,$0
2,2025,12%,>,"$11,601"
3,2025,22%,>,"$47,151"
4,2025,24%,>,"$100,526"
5,2025,32%,>,"$191,951"
6,2025,35%,>,"$243,726"
7,2025,37%,>,"$609,351"


In [154]:
merged_df = pd.concat([
    mfj.reset_index(drop=True),
    mfs.reset_index(drop=True),
    singles.reset_index(drop=True),
    hoh.reset_index(drop=True)
], axis=1)
merged_df

,Year,Married Filing Jointly (Rates/Brackets),For Income >,Range Start,Year,Married Filing Separately (Rates/Brackets),For Income >,Range Start,Year,Single Filer (Rates/Brackets),For Income >,Range Start,Year,Head of Household (Rates/Brackets),For Income >,Range Start
0,2025,10%,>,$0,2025,10%,>,$0,2025,10%,>,$0,2025,10%,>,$0
1,2025,12%,>,"$23,201",2025,12%,>,"$11,601",2025,12%,>,"$11,601",2025,12%,>,"$16,551"
2,2025,22%,>,"$94,301",2025,22%,>,"$47,151",2025,22%,>,"$47,151",2025,22%,>,"$63,101"
3,2025,24%,>,"$201,051",2025,24%,>,"$100,526",2025,24%,>,"$100,526",2025,24%,>,"$100,501"
4,2025,32%,>,"$383,901",2025,32%,>,"$191,951",2025,32%,>,"$191,951",2025,32%,>,"$191,951"
5,2025,35%,>,"$487,451",2025,35%,>,"$243,726",2025,35%,>,"$243,726",2025,35%,>,"$243,701"
6,2025,37%,>,"$731,201",2025,37%,>,"$365,601",2025,37%,>,"$609,351",2025,37%,>,"$609,351"


In [155]:
# Keep only the first 'Year' column, drop the rest
year_cols = [col for col in merged_df.columns if col == 'Year']
cols_to_drop = year_cols[1:]  # keep the first occurrence
merged_df = merged_df.drop(columns=cols_to_drop)
merged_df

,Married Filing Jointly (Rates/Brackets),For Income >,Range Start,Married Filing Separately (Rates/Brackets),For Income >,Range Start,Single Filer (Rates/Brackets),For Income >,Range Start,Head of Household (Rates/Brackets),For Income >,Range Start
0,10%,>,$0,10%,>,$0,10%,>,$0,10%,>,$0
1,12%,>,"$23,201",12%,>,"$11,601",12%,>,"$11,601",12%,>,"$16,551"
2,22%,>,"$94,301",22%,>,"$47,151",22%,>,"$47,151",22%,>,"$63,101"
3,24%,>,"$201,051",24%,>,"$100,526",24%,>,"$100,526",24%,>,"$100,501"
4,32%,>,"$383,901",32%,>,"$191,951",32%,>,"$191,951",32%,>,"$191,951"
5,35%,>,"$487,451",35%,>,"$243,726",35%,>,"$243,726",35%,>,"$243,701"
6,37%,>,"$731,201",37%,>,"$365,601",37%,>,"$609,351",37%,>,"$609,351"
